<h1 style="font-size:300%; background-color:powderblue; color:red; text-align:center; line-height : 60px;">
Tarea 1</h1>
<h3>Luis Carlos Diaz Fajardo</h3>
<h3>Diego Godoy</h3>

<h3>3.Desarrollar una función para rotar de imágenes, cuyo parámetro de entrada sea el ángulo de rotación y cuya salida sea la imagen rotada y debidamente interpolada.</h3>
<p>Para rotar las imagenes se usa la formula:</p>
<br>
$\left[ \begin{array}{c} x' \\ y' \end{array} \right] =
\left[ \begin{array}{cc} cos(\theta) & -sin(\theta) \\ sin(\theta) & cos(\theta) \end{array} \right]
\left[ \begin{array}{c} x \\ y \end{array} \right]$
<br>
<p>Donde a cada pixel se le asigna una nueva posición, debido a la naturaleza continua de las funciones trigonométricas algunos pixeles de la imagen de salida quedan vacios.</p>
<table>
    <tr>
        <th style="font-size:18px; text-align:center">Imagen original</th>
        <th style="font-size:18px; text-align:center">Sin interpolación</th>
    </tr>
    <tr>
        <td><img src=".\imgOriginl.jpg" width = "350"></td>
        <td><img src=".\imgRotNone.jpg" width = "350"></td>
    </tr>
</table>
<p>Para solucionar este inconveniente es preciso interpolar los espacios vacios, para estó se implementan dos interpolaciones. En primer lugar la interpolacion de vecino cercano asigna el valor del pixel mas cercano al espacio vacio, por otro lado la interpolación bilineal toma los cuatro pixeles que están alrededor del espacio vacio y realiza un promedio.</p>
<table>
    <tr>
        <th style="font-size:18px; text-align:center">Interpolación de vecino cercano</th>
        <th style="font-size:18px; text-align:center">Interpolación bilineal</th>
    </tr>
    <tr>
        <td><img src=".\imgRotNear.jpg" width = "350"></td>
        <td><img src=".\imgRotBili.jpg" width = "350"></td>
    </tr>
</table>
<p>A primera vista no hay mucha diferencia entre ellas, pero al hacer un acercamiento se puede ver que para el caso del vecino cercano la imagen es algo pixelada y los contornos no son suaves, en cambio para la imagen con interpolación lineal se pueden ver contornos suaves y el pixelado no es tan notorio.</p>
<table width="700" border="1" text-align="center">
    <tr>
        <th style="font-size:18px; text-align:center">Sin interpolación</th>
        <th style="font-size:18px; text-align:center">Interpolación de vecino cercano</th>
        <th style="font-size:18px; text-align:center">Interpolación bilineal</th>
    </tr>
    <tr>
        <td><img src=".\imgRotNoneZoom.jpg" width = "250"></td>
        <td><img src=".\imgRotNearZoom.jpg" width = "250"></td>
        <td><img src=".\imgRotBiliZoom.jpg" width = "250"></td>
    </tr>
</table>

In [ ]:
import cv2
from matplotlib import pyplot as plt
import numpy as np

def interpolate(image,x,y,method):
    if method=="nearest":
        image[x,y]=image[x,y-1]
    if method=="bilinear":
        suma=0
        cont=0
        for a in [-1,1]:
            if 0<x+a<image.shape[0]:
                suma+=image[x+a,y]
                cont+=1
            if 0<y+a<image.shape[1]:
                suma+=image[x,y+a]
                cont+=1
        image[x,y]=suma/cont
            
def rotateImage(inputImage, angle, interpolation):
    shape=inputImage.shape
    outputImage = np.zeros(shape)
    pixelCheck  = np.zeros(shape) 
    angleRad    = np.deg2rad(angle)
    
    for i in range(shape[0]):
        for j in range(shape[1]):
            x=i-shape[0]/2
            y=j-shape[1]/2
            
            xp=np.cos(angleRad)*x-np.sin(angleRad)*y
            yp=np.sin(angleRad)*x+np.cos(angleRad)*y
            
            xp=xp+shape[0]/2
            yp=yp+shape[1]/2
            
            xp=round(xp)
            yp=round(yp)
            
            if 0<=xp<shape[0] and 0<=yp<shape[1]:
                outputImage[xp,yp]=inputImage[i,j]
                pixelCheck[xp,yp] = 1      
                
    for i in range(shape[0]):
        for j in range(shape[1]):
            if pixelCheck[i,j]==0:
                validation=0
                for a in [-1,1]:
                    if 0<i+a<pixelCheck.shape[0] and pixelCheck[i+a,j]==1:
                        validation=1
                    if 0<j+a<pixelCheck.shape[1] and pixelCheck[i,j+a]==1:
                        validation=1
                if validation>0:
                    interpolate(outputImage,i,j,interpolation)
    return outputImage

img        = cv2.imread('img1.jpg',0)
imgRotNone = rotateImage(img,50,"none")
imgRotNear = rotateImage(img,50,"nearest")
imgRotBili = rotateImage(img,50,"bilinear")

cv2.imwrite("imgOriginl.jpg",img)
cv2.imwrite("imgRotNone.jpg",imgRotNone)
cv2.imwrite("imgRotNear.jpg",imgRotNear)
cv2.imwrite("imgRotBili.jpg",imgRotBili)

cv2.imwrite("imgRotNoneZoom.jpg",imgRotNone[100:150,100:150])
cv2.imwrite("imgRotNearZoom.jpg",imgRotNear[100:150,100:150])
cv2.imwrite("imgRotBiliZoom.jpg",imgRotBili[100:150,100:150])

plt.figure(1,figsize=[5, 5])
plt.imshow(img, cmap=plt.cm.gray)
plt.figure(2,figsize=[10, 10])
plt.imshow(imgRotNone, cmap=plt.cm.gray)
plt.figure(3,figsize=[10, 10])
plt.imshow(imgRotNear, cmap=plt.cm.gray)
plt.figure(4,figsize=[10, 10])
plt.imshow(imgRotBili, cmap=plt.cm.gray)